In [2]:
all_sentences = [
    "purple is the best city in the forest",
    "No way chimps go bananas for snacks!",
    "it is not often you find soggy bananas on the street",
    "green should have smelled more tranquil but somehow it just tasted rotten",
    "joyce enjoyed eating pancakes with ketchup",
    "throwing bananas on to the street is not art",
    "as the asteroid hurtled toward earth becky was upset her dentist appointment had been canceled",
    "I'm getting way too old. I don't even buy green bananas anymore.",
    "to get your way you must not bombard the road with yellow fruit",
    "Time flies like an arrow; fruit flies like a banana"
]

In [4]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('flax-sentence-embeddings/all_datasets_v3_mpnet-base')

# 10行のセンテンスを768次元のベクトルに変換（埋め込み）
all_embeddings = model.encode(all_sentences)
all_embeddings.shape

/opt/homebrew/Caskroom/miniforge/base/envs/vectordb/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
config_sentence_transformers.json: 100%|██████████| 116/116 [00:00<00:00, 129kB/s]
README.md: 100%|██████████| 9.85k/9.85k [00:00<00:00, 7.36MB/s]
sentence_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<00:00, 203kB/s]
config.json: 100%|██████████| 591/591 [00:00<00:00, 2.71MB/s]
pytorch_model.bin: 100%|██████████| 438M/438M [00:44<00:00, 9.92MB/s] 
/opt/homebrew/Caskroom/miniforge/base/envs/vectordb/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage(

(10, 768)

In [7]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
  (2): Normalize()
)

In [8]:
all_embeddings

array([[-0.01336706,  0.10373536, -0.00622339, ...,  0.03053999,
         0.01826637, -0.0170403 ],
       [ 0.0396864 , -0.00628764, -0.02050516, ...,  0.00605086,
        -0.05281233, -0.0341568 ],
       [ 0.05413862,  0.01059024, -0.01934125, ..., -0.0081793 ,
        -0.02009751,  0.00830053],
       ...,
       [ 0.04934242,  0.06136308,  0.01472958, ...,  0.01373632,
        -0.01050778, -0.00448923],
       [ 0.06331057,  0.09323005,  0.00229201, ..., -0.0064486 ,
         0.04311284, -0.02979367],
       [ 0.04161606,  0.03788033, -0.01343606, ...,  0.00287843,
         0.01492751, -0.03070068]], dtype=float32)

In [12]:
from transformers import AutoTokenizer

# 指定したtokenizerを用いる
# 今回はbert-base-uncasedを使用
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

all_tokens = [tokenizer.tokenize(sentence.lower()) for sentence in all_sentences]
all_tokens[0]

tokenizer_config.json: 100%|██████████| 28.0/28.0 [00:00<00:00, 118kB/s]
config.json: 100%|██████████| 570/570 [00:00<00:00, 1.85MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 705kB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 920kB/s]


['purple', 'is', 'the', 'best', 'city', 'in', 'the', 'forest']

In [13]:
all_tokens

[['purple', 'is', 'the', 'best', 'city', 'in', 'the', 'forest'],
 ['no', 'way', 'chi', '##mps', 'go', 'bananas', 'for', 'snacks', '!'],
 ['it',
  'is',
  'not',
  'often',
  'you',
  'find',
  'so',
  '##ggy',
  'bananas',
  'on',
  'the',
  'street'],
 ['green',
  'should',
  'have',
  'smelled',
  'more',
  'tran',
  '##quil',
  'but',
  'somehow',
  'it',
  'just',
  'tasted',
  'rotten'],
 ['joyce', 'enjoyed', 'eating', 'pancakes', 'with', 'ke', '##tch', '##up'],
 ['throwing', 'bananas', 'on', 'to', 'the', 'street', 'is', 'not', 'art'],
 ['as',
  'the',
  'asteroid',
  'hurt',
  '##led',
  'toward',
  'earth',
  'becky',
  'was',
  'upset',
  'her',
  'dentist',
  'appointment',
  'had',
  'been',
  'canceled'],
 ['i',
  "'",
  'm',
  'getting',
  'way',
  'too',
  'old',
  '.',
  'i',
  'don',
  "'",
  't',
  'even',
  'buy',
  'green',
  'bananas',
  'anymore',
  '.'],
 ['to',
  'get',
  'your',
  'way',
  'you',
  'must',
  'not',
  'bomb',
  '##ard',
  'the',
  'road',
  'with'

In [14]:
# Pineconeの初期化
from pinecone import Pinecone, PodSpec
pc = Pinecone(api_key='1a0e3dd8-12dd-42cc-92ef-16e20a38cb4b')

In [16]:
# インデックスの作成
# all_tokensを格納するDBの作成
pc.create_index(
    name="keyword-search-example", 
    dimension=all_embeddings.shape[1], 
    metric="euclidean",
    spec=PodSpec(
        environment='gcp-starter', 
        pod_type='starter'
    )
)

In [21]:
# インデックスに接続
index = pc.Index(name="keyword-search-example")

In [22]:
# all_embeddingsとall_tokensをInsertする

# all_embeddings...単語のベクトル埋め込み（数値）
# all_tokens...単語のベクトル
upserts = []
for i, (embedding, tokens) in enumerate(zip(all_embeddings, all_tokens)):
    upserts.append((str(i), embedding.tolist(), {'tokens': tokens}))

# 10行の文章が10個のベクトルとして格納される
index.upsert(vectors=upserts)

{'upserted_count': 10}

In [24]:
# セマンティック検索
query_sentence = "there is an art to getting your way and throwing bananas on to the street is not it"
xq = model.encode([query_sentence]).tolist()

result = index.query(vector=xq, top_k=5, includeMetadata=True)
result

{'matches': [{'id': '5',
              'metadata': {'tokens': ['throwing',
                                      'bananas',
                                      'on',
                                      'to',
                                      'the',
                                      'street',
                                      'is',
                                      'not',
                                      'art']},
              'score': 0.533911705,
              'values': []},
             {'id': '8',
              'metadata': {'tokens': ['to',
                                      'get',
                                      'your',
                                      'way',
                                      'you',
                                      'must',
                                      'not',
                                      'bomb',
                                      '##ard',
                                      'the',
               

In [27]:
# ハイブリッド検索（セマンティック検索＋キーワード検索）
# filterについて：https://docs.pinecone.io/docs/metadata-filtering
result = index.query(vector=xq, top_k=10, filter={'tokens': 'bananas'})
result

{'matches': [{'id': '5', 'score': 0.533911705, 'values': []},
             {'id': '2', 'score': 0.996979713, 'values': []},
             {'id': '1', 'score': 1.24630213, 'values': []},
             {'id': '7', 'score': 1.35194838, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 5}}

In [30]:
# 対象となるIDをもとに検索結果となるセンテンスを表示
ids = [int(match['id']) for match in result['matches']]
for i in ids:
    print(all_sentences[i])

throwing bananas on to the street is not art
it is not often you find soggy bananas on the street
No way chimps go bananas for snacks!
I'm getting way too old. I don't even buy green bananas anymore.
